In [1]:
import sys

sys.path.append('.')

In [2]:
import torch
from transformers import AutoTokenizer
from mamba_lm import from_pretrained
device = "cuda" if torch.cuda.is_available() else "cpu"

/Users/maciej/miniconda3/envs/llm-random/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = from_pretrained('state-spaces/mamba-130m').to(device)
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

/Users/maciej/miniconda3/envs/llm-random/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
x = model.forward(
    torch.tensor([tokenizer.encode("Hello, world!"), tokenizer.encode("Hello, world!")]))
#.unsqueeze(0).to(device)
# print(x)

x = model.forward_with_caches(
    torch.tensor([tokenizer.encode("Hello, world!"), tokenizer.encode("Hello, world!")]), requested_caches=[1, 1])
# .unsqueeze(0).to(device)
# print(x)

In [50]:
import copy

with torch.no_grad():
   model.eval()
   bs, seqlen = 10, 10
   split = 5

   requested_caches = [split] * bs
   input = torch.randint(0, 50256, (bs, seqlen)).to(device)
   x, our_caches = model.forward_with_caches(input, requested_caches=requested_caches)
   # for i in range(split+1, bs):
   #    logits, our_caches = model.step(input[:, i-1], caches=copy.deepcopy(our_caches))
   # final_logits = logits.clone()

   # reference_logits = model(input)
   reference_caches = [(None, torch.zeros(bs, model.config.d_inner, model.config.d_conv-1, device='cpu')) for _ in range(model.config.n_layers)]
   for i in range(split):
      logits_sequential, reference_caches = model.step(input[:, i], caches=reference_caches)

   for i in range(model.config.n_layers):
      assert (our_caches[i][0] - reference_caches[i][0]).norm() < 1e-3
      assert (our_caches[i][1] - reference_caches[i][1]).norm() < 1e-3


   logits_parallel = model(input)


   # print(torch.allclose(final_logits, reference_logits[:, split]))

   # input = torch.tensor([tokenizer.encode("Hello, world!"), tokenizer.encode("Hello, world!")])
   # x, our_caches = model.forward_with_caches(
   #    input[:, 0:1], requested_caches=[1, 1])
   # # our_caches = copy.deepcopy(our_caches)

   # zeros = [(None, torch.zeros(2, model.config.d_inner, model.config.d_conv-1, device='cpu')) for _ in range(model.config.n_layers)]
   # _, reference_caches = model.step(
   #     torch.tensor([tokenizer.encode("Hello, world!"), tokenizer.encode("Hello, world!")])[:, 0], caches=zeros)


In [56]:
(logits_parallel[:, split-1] - logits_sequential).abs().max()

tensor(0.0007)

In [37]:
# reference_logits[:, -1:, :].shape
# final_logits.shape
our_caches[0][0] - reference_caches[0][0]
# reference_logits - final_logits

tensor([[[-4.6566e-10, -9.3132e-10,  0.0000e+00,  ...,  3.7253e-09,
          -7.4506e-09,  1.8626e-09],
         [ 1.8626e-09, -2.9104e-11, -4.6566e-10,  ..., -2.3283e-10,
           2.3283e-10, -5.5879e-09],
         [ 2.3283e-10,  0.0000e+00, -2.9802e-08,  ..., -1.8626e-09,
           0.0000e+00, -9.3132e-10],
         ...,
         [ 1.8626e-09,  1.8626e-09, -1.4901e-08,  ...,  0.0000e+00,
          -2.2352e-08,  7.4506e-09],
         [ 1.4901e-08, -2.9802e-08, -2.3283e-10,  ..., -5.5879e-09,
           0.0000e+00,  0.0000e+00],
         [ 4.6566e-10,  0.0000e+00, -3.7253e-09,  ...,  0.0000e+00,
           0.0000e+00, -6.9849e-10]],

        [[ 0.0000e+00, -2.3283e-10,  2.9802e-08,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-4.6566e-10, -2.9104e-11,  0.0000e+00,  ..., -1.8626e-09,
           0.0000e+00,  9.3132e-10],
         [ 9.3132e-10,  3.7253e-09, -1.4901e-08,  ..., -3.7253e-09,
           1.4901e-08, -1.8626e-09],
         ...,
         [-3.7253e-09,  2

In [16]:
for layer in range(model.config.n_layers):
    for i in range(2):
        print((our_caches[layer][i] - reference_caches[layer][i]).norm())

tensor(3.4919e-06, grad_fn=<LinalgVectorNormBackward0>)
tensor(0., grad_fn=<LinalgVectorNormBackward0>)
tensor(9.2723e-07, grad_fn=<LinalgVectorNormBackward0>)
tensor(2.0019e-05, grad_fn=<LinalgVectorNormBackward0>)
tensor(4.7374e-07, grad_fn=<LinalgVectorNormBackward0>)
tensor(2.2792e-05, grad_fn=<LinalgVectorNormBackward0>)
tensor(3.5112e-07, grad_fn=<LinalgVectorNormBackward0>)
tensor(2.8428e-05, grad_fn=<LinalgVectorNormBackward0>)
tensor(1.3595e-06, grad_fn=<LinalgVectorNormBackward0>)
tensor(2.8514e-05, grad_fn=<LinalgVectorNormBackward0>)
tensor(1.0204e-06, grad_fn=<LinalgVectorNormBackward0>)
tensor(3.1822e-05, grad_fn=<LinalgVectorNormBackward0>)
tensor(1.6732e-06, grad_fn=<LinalgVectorNormBackward0>)
tensor(3.1905e-05, grad_fn=<LinalgVectorNormBackward0>)
tensor(2.0882e-06, grad_fn=<LinalgVectorNormBackward0>)
tensor(3.0574e-05, grad_fn=<LinalgVectorNormBackward0>)
tensor(1.7318e-06, grad_fn=<LinalgVectorNormBackward0>)
tensor(3.0373e-05, grad_fn=<LinalgVectorNormBackward0>)


24

In [8]:
x = model.step(
    torch.tensor([tokenizer.encode("Hello, world!"), tokenizer.encode("Hello, world!")]), caches=[None, None])

TypeError: cannot unpack non-iterable NoneType object